### Load Package

In [1]:
import pandas as pd
import mne
import os
import numpy as np
import pickle
import torch
from mne.preprocessing import ICA, create_eog_epochs
from mne_icalabel import label_components

### Global Variable

In [2]:
base_path = '../data/'

sfreq = 500  
l_freq = 2.0  
h_freq = 50.0  

subjects_data = {}

### Load Channel Names

In [ ]:
eloc30_path = "../電極通道/BD_MDD_HC_30.loc"
eloc30 = mne.channels.read_custom_montage(eloc30_path)

ch_names = eloc30.ch_names

### Prepocessing

In [4]:
processed_data = {}
success_count = 0
error_count = 0
error_files = []

for root, dirs, files in os.walk(base_path):
    subject_files = [f for f in files if f.endswith('_Filter.csv') or f.endswith('_Filter_fix.csv')]
    if subject_files:
        subject_id = os.path.basename(root)
        
        for file in subject_files:
            file_path = os.path.join(root, file)

            try:
                data = pd.read_csv(file_path, header=None, dtype=np.float64)
                print('=========================================================================================================================')
                print('=========================================================================================================================')
                print('Data shape before checking: ', data.shape)

                # check if the last column is all NaN
                if data.iloc[:, -1].isna().all():
                    data = data.iloc[:, :-1]


                data = data.values
                print('Data shape after checking: ', data.shape)

                # process nan & inf
                if np.isnan(data).any() or np.isinf(data).any():
                    print(f'Found NaN or inf in {file}, applying nan_to_num.')
                    data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)



                # remove the 16 and the 32 row
                data = np.delete(data, [15, 31], axis=0)


                # Cz reference
                ref_data = data[21]  
                data = data - ref_data  
                data = np.delete(data, 21, axis=0)
                ch_names_reduced = np.delete(ch_names, 21)



                # create MNE Info object
                info = mne.create_info(ch_names=list(ch_names_reduced), sfreq=sfreq, ch_types='eeg')

                # change data to mne.RawArray
                raw = mne.io.RawArray(data, info)

                # set channel locations
                raw.set_montage(eloc30)

                # digital filter(2~50Hz)
                raw.filter(l_freq=2.0, h_freq=50.0)

                # ICA with Infomax
                ica = ICA(n_components=len(ch_names_reduced), method='infomax', fit_params=dict(extended=True), random_state=97)
                ica.fit(raw)

                # Use ICLabel for automatic component labeling and classification
                labels = label_components(raw, ica, method='iclabel')

                # Get the labeled components
                label_names = labels['labels']  
                label_probs = labels['y_pred_proba'] 

                # Set thresholds and exclude components of specific categories
                exclude_ics = []
                for idx, (name, probs) in enumerate(zip(label_names, label_probs)):
                    if (name == 'muscle artifact' and probs.max() >= 0.7) or \
                       (name == 'eye' and probs.max() >= 0.7) or \
                       (name == 'brain' and probs.min() <= 0.2) or \
                       (name == 'other' and probs.max() >= 0.7):
                        exclude_ics.append(idx)

                # Exclude selected components
                ica.exclude = exclude_ics
                raw_corrected = ica.apply(raw)

                # Save the processed data as a NumPy array
                processed_data[(subject_id, file)] = raw_corrected.get_data()
                print(f'Processed successfully.')
                print(f'Processed data shape: {processed_data[(subject_id, file)].shape}')
                success_count += 1

            except Exception as e:
                print(f'Error processing {subject_id} - File {file}: {e}')
                error_count += 1
                error_files.append((subject_id, file))
                continue

print("Finished processing all files.")
print(f"Successfully processed {success_count} files.")
print(f"Failed to process {error_count} files.")

Data shape before checking:  (32, 46001)
Data shape after checking:  (32, 46000)
Creating RawArray with float64 data, n_channels=29, n_times=46000
    Range : 0 ... 45999 =      0.000 ...    91.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (1.650 s)

Fitting ICA to data using 29 channels (please be patient, this may take a while)
Selecting by number: 29 components
Computing Extended Infomax ICA
Fitting ICA took 24.4s.


C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')
C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')


Applying ICA to Raw instance
    Transforming to ICA space (29 components)
    Zeroing out 14 ICA components
    Projecting back using 29 PCA components
Processed successfully.
Processed data shape: (29, 46000)
Data shape before checking:  (32, 46001)
Data shape after checking:  (32, 46000)
Creating RawArray with float64 data, n_channels=29, n_times=46000
    Range : 0 ... 45999 =      0.000 ...    91.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (1.650 s)

Fitting ICA to dat

C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')
C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')


Applying ICA to Raw instance
    Transforming to ICA space (29 components)
    Zeroing out 19 ICA components
    Projecting back using 29 PCA components
Processed successfully.
Processed data shape: (29, 46000)
Data shape before checking:  (32, 46001)
Data shape after checking:  (32, 46000)
Creating RawArray with float64 data, n_channels=29, n_times=46000
    Range : 0 ... 45999 =      0.000 ...    91.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (1.650 s)

Fitting ICA to dat

...

Applying ICA to Raw instance
    Transforming to ICA space (29 components)
    Zeroing out 8 ICA components
    Projecting back using 29 PCA components
Processed successfully.
Processed data shape: (29, 46000)
Data shape before checking:  (32, 46001)
Data shape after checking:  (32, 46000)
Creating RawArray with float64 data, n_channels=29, n_times=46000
    Range : 0 ... 45999 =      0.000 ...    91.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (1.650 s)

Fitting ICA to data

C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')
C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')


Applying ICA to Raw instance
    Transforming to ICA space (29 components)
    Zeroing out 6 ICA components
    Projecting back using 29 PCA components
Processed successfully.
Processed data shape: (29, 46000)
Data shape before checking:  (32, 46001)
Data shape after checking:  (32, 46000)
Creating RawArray with float64 data, n_channels=29, n_times=46000
    Range : 0 ... 45999 =      0.000 ...    91.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (1.650 s)

Fitting ICA to data

C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')
C:\Users\NESS-Kuan\AppData\Local\Temp\ipykernel_47528\2555352665.py:64: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  labels = label_components(raw, ica, method='iclabel')


Applying ICA to Raw instance
    Transforming to ICA space (29 components)
    Zeroing out 2 ICA components
    Projecting back using 29 PCA components
Processed successfully.
Processed data shape: (29, 46000)
Finished processing all files.
Successfully processed 30 files.
Failed to process 0 files.


### Save Data

In [ ]:
output_file = '../data/BD5_processed_data.pkl'


with open(output_file, 'wb') as f:
    pickle.dump(processed_data, f)

print(f'processed_data has been saved to {output_file}')

processed_data has been saved to ../data/BD5_processed_data.pkl


: 